#### Data Cleaning --Remove NaN Values from the JSON

In [ ]:
import pandas as pd
import json
# Path to your JSON file
file_path = r'C:\Users\saswa\OneDrive\Documents\Puffy\sample_events_data.json'
# Read the JSON file into a DataFrame
df = pd.read_json(file_path)
# Replace all NaN with None (so that they become 'null' in JSON)
df_clean = df.where(pd.notnull(df), None)
# Optional: Save the cleaned version back to file (for SQL Server OPENROWSET)
cleaned_path = r'C:\Users\saswa\OneDrive\Documents\Puffy\sample_events_data_clean.json'
df_clean.to_json(cleaned_path, orient='records', indent=2)
with open(cleaned_path, 'r') as f:
    json_data = json.load(f)  # Will raise error if not valid
print("✅ JSON is valid")


#### Connect to SQL Server

In [71]:
import pyodbc
SERVER_NAME = 'Saswat\\SQLEXPRESS'
DATABASE_NAME = 'msdb'  # Or the specific database you want to connect to
connection_string = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={SERVER_NAME};"
    f"DATABASE={DATABASE_NAME};"
    f"Trusted_Connection=yes;"
)
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()
print("✅ Connected to SQL Server successfully!")

✅ Connected to SQL Server successfully!


#### Load Cleaned JSON into a Table PuffyRawDataEvents

In [72]:
# SQL query to Read the cleansed JSON and Load to an SQL Server table named PuffyRawDataEvents
sql_query = """
SELECT
    event_timestamp,
    event_name,
    event_previous_timestamp,
    event_value_in_usd,
    event_bundle_sequence_id,
    event_server_timestamp_offset,
    user_id,
    user_pseudo_id,
    traffic_source_name,
    traffic_source_medium,
    traffic_source_source,
    platform,
    JSON_VALUE(event_params, '$.ga_session_number') AS ga_session_number,
    JSON_VALUE(event_params, '$.campaign') AS campaign,
    JSON_VALUE(event_params, '$.content') AS content,
    JSON_VALUE(event_params, '$.gclid') AS gclid,
    JSON_VALUE(event_params, '$.value') AS value,
    JSON_VALUE(event_params, '$.tax') AS tax,
    JSON_VALUE(event_params, '$.srsltid') AS srsltid,
    JSON_VALUE(event_params, '$.ga_session_id') AS ga_session_id,
    JSON_VALUE(event_params, '$.session_engaged') AS session_engaged,
    JSON_VALUE(event_params, '$.page_referrer') AS page_referrer,
    JSON_VALUE(event_params, '$.engagement_time_msec') AS engagement_time_msec,
    JSON_VALUE(event_params, '$.entrances') AS entrances,
    JSON_VALUE(event_params, '$.ecomm_pagetype') AS ecomm_pagetype,
    JSON_VALUE(event_params, '$.currency') AS currency,
    JSON_VALUE(event_params, '$.term') AS term,
    JSON_VALUE(event_params, '$.source') AS source,
    JSON_VALUE(event_params, '$.ecomm_totalvalue') AS ecomm_totalvalue,
    JSON_VALUE(event_params, '$.ecomm_prodid') AS ecomm_prodid,
    JSON_VALUE(event_params, '$.engaged_session_event') AS engaged_session_event,
    JSON_VALUE(event_params, '$.shipping') AS shipping,
    JSON_VALUE(event_params, '$.medium') AS medium,
    JSON_VALUE(event_params, '$.ignore_referrer') AS ignore_referrer,
    JSON_VALUE(event_params, '$.transaction_id') AS transaction_id
INTO PuffyRawDataEvents
FROM OPENROWSET(
    BULK 'C:\\Users\\saswa\\OneDrive\\Documents\\Puffy\\sample_events_data_clean.json',
    SINGLE_CLOB
) AS raw
CROSS APPLY OPENJSON(BulkColumn)
WITH (
    event_timestamp bigint,
    event_name nvarchar(100),
    event_previous_timestamp nvarchar(100),
    event_value_in_usd nvarchar(100),
    event_bundle_sequence_id bigint,
    event_server_timestamp_offset nvarchar(100),
    user_id nvarchar(100),
    user_pseudo_id nvarchar(100),
    traffic_source_name nvarchar(100),
    traffic_source_medium nvarchar(100),
    traffic_source_source nvarchar(100),
    platform nvarchar(100),
    event_params nvarchar(max) AS JSON
) AS evt;
"""

# Attempt to execute query
try:
    with pyodbc.connect(connection_string) as conn:
        with conn.cursor() as cursor:
            cursor.execute(sql_query)
            conn.commit()
    print("SQL query executed successfully.")
except Exception as e:
    print("Error executing SQL query:", e)



Error executing SQL query: ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'PuffyRawDataEvents' in the database. (2714) (SQLExecDirectW)")


#### Check Unique Event Names in the Dataset

In [73]:
# Let's also check what unique event names exist in the dataset
query_events = """
SELECT DISTINCT event_name, COUNT(*) as event_count
FROM PuffyRawDataEvents
GROUP BY event_name
ORDER BY event_count DESC
"""

try:
    cursor.execute(query_events)
    rows = cursor.fetchall()
    columns = [column[0] for column in cursor.description]
    events_df = pd.DataFrame.from_records(rows, columns=columns)

    print("Available event types:")
    display(events_df)

except pyodbc.Error as e:
    print(f"Error: {str(e)}")

Available event types:


,event_name,event_count
0,page_view,10056
1,view_item,5664
2,session_start,5431
3,first_visit,3802
4,add_to_cart,857
5,begin_checkout,268
6,purchase,88


#### Check the Date Range in the Dataset

In [74]:
# Check the date range in the dataset
query_date_range = """
SELECT
    MIN(CONVERT(date, DATEADD(SECOND, event_timestamp / 1000000, '1970-01-01'))) as min_date,
    MAX(CONVERT(date, DATEADD(SECOND, event_timestamp / 1000000, '1970-01-01'))) as max_date,
    COUNT(DISTINCT CONVERT(date, DATEADD(SECOND, event_timestamp / 1000000, '1970-01-01'))) as total_days,
    COUNT(*) as total_events
FROM PuffyRawDataEvents
"""

try:
    cursor.execute(query_date_range)
    rows = cursor.fetchall()
    columns = [column[0] for column in cursor.description]
    date_range_df = pd.DataFrame.from_records(rows, columns=columns)

    print("Dataset date range:")
    display(date_range_df)

except pyodbc.Error as e:
    print(f"Error: {str(e)}")

Dataset date range:


,min_date,max_date,total_days,total_events
0,2023-05-30,2023-06-13,15,26166


#### Query for Comprehensive Funnel Segment Analysis

In [75]:
# Comprehensive e-commerce funnel analysis query
funnel_query = """
WITH daily_funnel AS (
    SELECT
        CONVERT(date, DATEADD(SECOND, event_timestamp / 1000000, '1970-01-01')) as event_date,
        user_pseudo_id,
        -- Define funnel stages based on common e-commerce events
        MAX(CASE WHEN event_name IN ('page_view', 'session_start', 'user_engagement') THEN 1 ELSE 0 END) as visited_site,
        MAX(CASE WHEN event_name IN ('view_item', 'product_view', 'view_item_list') THEN 1 ELSE 0 END) as viewed_product,
        MAX(CASE WHEN event_name IN ('add_to_cart', 'add_to_wishlist') THEN 1 ELSE 0 END) as added_to_cart,
        MAX(CASE WHEN event_name IN ('begin_checkout', 'checkout_progress', 'add_payment_info', 'add_shipping_info') THEN 1 ELSE 0 END) as initiated_checkout,
        MAX(CASE WHEN event_name IN ('purchase', 'ecommerce_purchase', 'transaction') THEN 1 ELSE 0 END) as completed_purchase
    FROM PuffyRawDataEvents
    WHERE CONVERT(date, DATEADD(SECOND, event_timestamp / 1000000, '1970-01-01')) IS NOT NULL
    GROUP BY CONVERT(date, DATEADD(SECOND, event_timestamp / 1000000, '1970-01-01')), user_pseudo_id
),
funnel_metrics AS (
    SELECT
        event_date,
        -- Count users at each funnel stage
        COUNT(DISTINCT user_pseudo_id) as total_users,
        COUNT(DISTINCT CASE WHEN visited_site = 1 THEN user_pseudo_id END) as users_visited_site,
        COUNT(DISTINCT CASE WHEN viewed_product = 1 THEN user_pseudo_id END) as users_viewed_product,
        COUNT(DISTINCT CASE WHEN added_to_cart = 1 THEN user_pseudo_id END) as users_added_to_cart,
        COUNT(DISTINCT CASE WHEN initiated_checkout = 1 THEN user_pseudo_id END) as users_initiated_checkout,
        COUNT(DISTINCT CASE WHEN completed_purchase = 1 THEN user_pseudo_id END) as users_completed_purchase
    FROM daily_funnel
    GROUP BY event_date
)
SELECT
    event_date,
    users_visited_site as stage_1_site_visit,
    users_viewed_product as stage_2_product_view,
    users_added_to_cart as stage_3_add_to_cart,
    users_initiated_checkout as stage_4_checkout,
    users_completed_purchase as stage_5_purchase,

    -- Calculate step-wise conversion rates
    CASE
        WHEN users_visited_site > 0
        THEN ROUND(CAST(users_viewed_product AS FLOAT) / users_visited_site * 100, 2)
        ELSE 0
    END as conv_rate_visit_to_product_view,

    CASE
        WHEN users_viewed_product > 0
        THEN ROUND(CAST(users_added_to_cart AS FLOAT) / users_viewed_product * 100, 2)
        ELSE 0
    END as conv_rate_product_to_cart,

    CASE
        WHEN users_added_to_cart > 0
        THEN ROUND(CAST(users_initiated_checkout AS FLOAT) / users_added_to_cart * 100, 2)
        ELSE 0
    END as conv_rate_cart_to_checkout,

    CASE
        WHEN users_initiated_checkout > 0
        THEN ROUND(CAST(users_completed_purchase AS FLOAT) / users_initiated_checkout * 100, 2)
        ELSE 0
    END as conv_rate_checkout_to_purchase,

    -- Overall conversion rate from site visit to purchase
    CASE
        WHEN users_visited_site > 0
        THEN ROUND(CAST(users_completed_purchase AS FLOAT) / users_visited_site * 100, 2)
        ELSE 0
    END as overall_conversion_rate

FROM funnel_metrics
WHERE event_date IS NOT NULL
ORDER BY event_date;
"""
try:
    cursor.execute(funnel_query)
    rows = cursor.fetchall()
    columns = [column[0] for column in cursor.description]
    funnel_df = pd.DataFrame.from_records(rows, columns=columns)

# Safe CSV saving with error handling
    import os
    from datetime import datetime

    csv_filename = r'C:\Users\saswa\OneDrive\Documents\Puffy\ecommerce_funnel_analysis.csv'

    try:
        funnel_df.to_csv(csv_filename, index=False)
        print(f"\n✅ Results saved to '{csv_filename}'")
    except PermissionError:
        # Try alternative filename with timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        alternative_filename = f'ecommerce_funnel_analysis_{timestamp}.csv'
        try:
            funnel_df.to_csv(alternative_filename, index=False)
            print(f"\n✅ Results saved to '{alternative_filename}' (original file was locked)")
        except PermissionError:
            # Try saving to user's Documents folder
            try:
                documents_path = os.path.join(os.path.expanduser("~"), "Documents")
                fallback_filename = os.path.join(documents_path, alternative_filename)
                funnel_df.to_csv(fallback_filename, index=False)
                print(f"\n✅ Results saved to '{fallback_filename}'")
            except Exception as e:
                print(f"\n⚠️ Could not save CSV file: {e}")
                print("Data is still available in the 'funnel_df' variable for further analysis.")

except pyodbc.Error as e:
    print(f"Error executing funnel query: {str(e)}")



✅ Results saved to 'C:\Users\saswa\OneDrive\Documents\Puffy\ecommerce_funnel_analysis.csv'


In [76]:
# Calculate overall funnel summary
if 'funnel_df' in locals() and not funnel_df.empty:
    print("\n📊 Funnel Summary Statistics:")
    print(f"Total days analyzed: {len(funnel_df)}")
    print(f"Average daily site visits: {funnel_df['stage_1_site_visit'].mean():.0f}")
    print(f"Average overall conversion rate: {funnel_df['overall_conversion_rate'].mean():.2f}%")
    print(f"Best performing day: {funnel_df.loc[funnel_df['overall_conversion_rate'].idxmax(), 'event_date']} ({funnel_df['overall_conversion_rate'].max():.2f}%)")


📊 Funnel Summary Statistics:
Total days analyzed: 15
Average daily site visits: 360
Average overall conversion rate: 1.67%
Best performing day: 2023-06-10 (3.53%)
